# Import librairies

In [44]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import folium

In [3]:
from modules.connecting_to_bdd import fetch_table_from_trafic_db
from modules.geodata import convert_to_linestring, has_close_event

# Import dataset

In [4]:
evenements_df = fetch_table_from_trafic_db('events_data')

In [5]:
trafic_df = fetch_table_from_trafic_db('trafic_routier')

In [6]:
trafic_df.head()

,id_technique,id,debit,longueur,taux_occupation,code_couleur,nom_du_troncon,etat_du_trafic,temps_de_parcours,vitesse,geo_point_2d,geometrie,shape_geo,horodatage,type_geo,coordinates_geo
0,772-20250218T205200,772,120,410,1.2,3,Vannes I9,Fluide,62,24,"{47.23414730688424,-1.5808786419612229}","{{-1.583123434893785,47.235185973808406},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5808786419612229,47.23414730688424}"
1,158-20250218T205200,158,300,364,3.7,3,Le Lasseur P2,Fluide,63,21,"{47.23083661767159,-1.5704468983375661}","{{-1.568589501648445,47.23188191655151},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.5704468983375661,47.23083661767159}"
2,497-20250218T205200,497,180,224,2.0,3,Lauriol I4,Fluide,37,22,"{47.23531301184315,-1.5635261418937887}","{{-1.563539784805616,47.236320687488636},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5635261418937887,47.23531301184315}"
3,5444-20250218T205200,5444,240,177,2.9,3,Pasteur P1,Fluide,40,16,"{47.21255561611825,-1.579953397082114}","{{-1.581033846171769,47.21286509290865},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.579953397082114,47.21255561611825}"
4,5053-20250218T205200,5053,60,224,0.8,3,Barbusse P2,Fluide,44,18,"{47.22705355840811,-1.5524880999813486}","{{-1.551553912867712,47.22760386550291},{-1.55...",LineString,2025-02-18 20:52:00,Point,"{-1.5524880999813486,47.22705355840811}"


In [7]:
trafic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554761 entries, 0 to 1554760
Data columns (total 16 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   id_technique       1554761 non-null  object        
 1   id                 1554761 non-null  int64         
 2   debit              1554761 non-null  int64         
 3   longueur           1554761 non-null  int64         
 4   taux_occupation    1554761 non-null  float64       
 5   code_couleur       1554761 non-null  int64         
 6   nom_du_troncon     1554761 non-null  object        
 7   etat_du_trafic     1554761 non-null  object        
 8   temps_de_parcours  1554761 non-null  int64         
 9   vitesse            1554761 non-null  int64         
 10  geo_point_2d       1554761 non-null  object        
 11  geometrie          1554761 non-null  object        
 12  shape_geo          1554761 non-null  object        
 13  horodatage         1554761 

# Jointure

In [8]:
# Convertir les donnée en type LineString
trafic_df['geometrie_linestring'] = trafic_df['geometrie'].apply(convert_to_linestring)

In [9]:
# Convertir la colonne 'coordinates_geo' en objets Point
evenements_df['geometrie_point'] = evenements_df['coordinates_geo'].apply(lambda x: Point(eval(x)))  

In [10]:
# Convertir les DataFrames en GeoDataFrames
trafic_gdf = gpd.GeoDataFrame(trafic_df, geometry='geometrie_linestring', crs="EPSG:4326")  # Système de coordonnées WGS 84
evenements_gdf = gpd.GeoDataFrame(evenements_df, geometry='geometrie_point', crs="EPSG:4326")

In [ ]:
# Faire un dataframe avec les troncon_unique
unique_troncon = trafic_gdf.drop_duplicates(subset='nom_du_troncon', keep='first')

In [14]:
# Faire un dataframe avec les events unique
unique_event_place = evenements_gdf.drop_duplicates(subset='location_name', keep='first')

In [15]:
unique_event_place['location_name'].value_counts()

location_name
Stade de la Beaujoire Louis Fonteneau    1
Cité internationale des Congrès          1
Palais des Sports Beaulieu               1
Parc des Expositions de la Beaujoire     1
STEREOLUX                                1
Zénith de Nantes Métropole               1
Name: count, dtype: int64

## Représentation graphique

In [47]:
# Créer une carte centrée sur Nantes (coordonnées de Nantes)
map_nantes = folium.Map(location=[47.2184, -1.5536], zoom_start=13)

# # Ajouter tous les tronçons de trafic sous forme de lignes (LineString)
for idx, row in unique_troncon.iterrows():
    # S'assurer que la géométrie est un LineString
    if isinstance(row['geometrie_linestring'], LineString):
        # Convertir les coordonnées du LineString pour Folium
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in row['geometrie_linestring'].coords], 
            color='blue',  # Couleur de la ligne
            weight=2.5,  # Épaisseur de la ligne
            opacity=1
        ).add_to(map_nantes)

# Ajouter tous les événements sous forme de points (Point)
for idx, event in unique_event_place.iterrows():
    # S'assurer que la géométrie de l'événement est de type Point
    event_geom = event['geometrie_point']
    
    if isinstance(event_geom, Point):
        # Ajouter un point (Marker) pour chaque événement directement
        folium.Marker(
            location=[event_geom.x, event_geom.y],
            popup=event['location_name'],  
            icon=folium.Icon(color='red')  
        ).add_to(map_nantes)

# Afficher la carte
map_nantes
